In [ ]:
import requests
import pandas as pd

In [ ]:
df = pd.DataFrame.from_csv(".csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
params = {
    "action":"query",
    "prop":"revisions",
    "format":"json",
    "rvlimit":"max",
    "rvprop":"ids|timestamp|user",
    "rvdir":"newer"
}

In [ ]:
def get_revisions(row):
    params["rvstart"]="{}T00:00:00Z".format(row['year_month']),
    params["rvend"]="{}-31T23:59:59Z".format("-".join(row['year_month'].split('-')[:2])),
    params["rvuser"]=row['editor_name']
    params["pageids"]=row['article_id']
    r = requests.get("https://de.wikipedia.org/w/api.php", params=params)
    if 'revisions' in r.json()['query']['pages'][str(row['article_id'])]:
        return [revid['revid'] for revid in r.json()['query']['pages'][str(row['article_id'])]['revisions']]
    else:
        return "no revisions"

In [ ]:
%%time
df['revisions'] = df[['year_month', 'editor_name', 'article_id']].apply(get_revisions, axis=1)

In [ ]:
%%time
df['num_edits'] = df['revisions'].apply(len) 

In [ ]:
df.set_index('index', inplace=True)

In [ ]:
df.to_csv('.csv')